# Train and test an SVM

## Import boilerplate and setup

In [1]:
# first, change the working directory, if needed
import os
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
os.getcwd()

'/mnt/c/Users/aryam/Documents/school/2020/spring/cse283/BloodBrainClassifier'

In [2]:
# now, import whatever else we need
from sklearn.svm import SVC
import scripts.starter as data
from sklearn.metrics import accuracy_score

## Train SVC model

In [3]:
clf = SVC()
clf.fit(data.X_train, data.y_train)

SVC()

## Test SVC model

In [4]:
y_pred = clf.predict(data.X_test)

## Evaluate the model's performance

In [5]:
accuracy_score(data.y_test, y_pred)

0.6052631578947368

In [ ]:
# lol, that's pretty terrible
# TODO: improve the model